#### 节点为 HH, 使用切割法实现螺旋波

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from numba import njit, prange
from import_fun import HH, Diffusion2D, show_spiral_wave

In [2]:
# 开启交互式画图
mpl.use('TkAgg') # TkAgg Qt5Agg
plt.ion()  # 开启交互模式

In [3]:
Nx = 100
Ny = 100
method = "euler"
dt = 0.01

In [4]:
# 生成节点，初始值设定
nodes = HH(N=Nx*Ny, method=method, dt=dt)
nodes.vars_nodes[0] = 0.
nodes.vars_nodes[1] = 0.
nodes.vars_nodes[2] = 0.
nodes.vars_nodes[3] = 0.
nodes.params_nodes["Iex"] = 0.
nodes.spiking = False    # 关掉峰值探测器

# 生成一个方阵视图，这个视图与原始数组共享内存
V_view = nodes.vars_nodes[0].reshape(Nx, Ny)
m_view = nodes.vars_nodes[1].reshape(Nx, Ny)
h_view = nodes.vars_nodes[2].reshape(Nx, Ny)
n_view = nodes.vars_nodes[3].reshape(Nx, Ny)

I_input = np.zeros(Nx*Ny)
I_input_view = I_input.reshape(Nx, Ny)


In [5]:
# 设定动态显示器
spiral_wave = show_spiral_wave(nodes.vars_nodes[0], Nx, Ny, save_gif=True)

In [6]:
# 设定扩散耦合
## "No_flow", "periodic"
syn = Diffusion2D(D=0.6, boundary="No_flow", adjacency=8)

In [7]:
# mem_list = []
for i in range(100_00):
    nodes()

nodes.t = 0.
# I_input_view[:, 0] = 10

mod = int(Nx/2)
for i in range(60_00):
    if i<10_00:
        I_input_view[:, 0] = 10
    else:
        I_input_view[:, 0] = 0.

    if i == 22_00:
        V_view[mod:, :] = -65.00095684
        m_view[mod:, :] = 0.05292585
        h_view[mod:, :] = 0.59450462
        n_view[mod:, :] = 0.31761895

    spiral_wave(i, nodes.t, show_interval=1_00)
    Isyn = syn(V_view)
    nodes(Isyn+I_input)

In [8]:
spiral_wave.save_image(filename="animation.gif", duration=50)